In [82]:
import os
from langchain_mistralai import ChatMistralAI
from langchain.schema import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings
from dotenv import load_dotenv
import time
from pinecone import Pinecone as pc
from langchain_pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from tqdm.autonotebook import tqdm
from langchain_core.tools import tool
import env

# importing the py files
from user_profile import *

In [80]:
llm = ChatMistralAI(model="mistral-large-latest")

In [83]:
user_profile = {
    "Name": "John Doe",
    "Age": 70,
    "Gender": "Male",
    "Location": "New York, NY, USA",
    "Account Age": "3 years",
    "Visit Frequency": "10 visits in the last 10 days",
    "Purchase Frequency": "5 purchases in the last month",
    "Average Purchase Value": "$50 per purchase",
    "Cart Abandonment Rate": "3 abandoned carts in the last month",
    "Engagement with Promotions": "Clicked on 3 promotional emails in the last month",
    "Wishlist Activity": "Added 5 items to wishlist in the last month",
    "Browsing History": ["Laptops", "Smartphones", "Books"],
    "Subscription Status": "No",
    "Preferred Payment Methods": ["Credit Card", "Mobile Wallet"]
}

user_train = user_profile_train(user_profile)
user_needs = user_train.get_user_profile()

# print(user)

needs = ""
for i in user_needs:
    needs = needs+i+"\n"
print(needs)

c:\Users\aichr\.virtualenvs\Amazon-HackOn-vcHtwuFW\lib\site-packages\langchain_mistralai\embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


Simplified
secure payment interfaces
Tools to manage fixed incomes and budgeting
Fraud protection and financial advice
Innovative payment solutions (e.g.
mobile wallets
buy now
pay later options)
Financial education and budgeting tools
Easy-to-use payment options
Loyalty rewards and cashback offers
Budget tracking and management tools



In [46]:
query_category = {
    "Payment Failure": "High Pain, High Importance",
    "Order Confirmation": "High Pain, High Importance",
    "Delayed Refunds": "High Pain, High Importance",
    "Security Concern": "High Pain, High Importance",
    "International Payments": "High Pain, Low Importance",
    "Hidden Fees": "High Pain, Low Importance",
    "Transaction Limit": "Low Pain, Low Importance",
    "Loyalty Points and Gift cards": "Low Pain, Low Importance",
    "Promo codes and discount issues": "Low Pain, Low Importance",
    "Complex checkout process": "Low Pain, High Importance",
    "Payment options": "Low Pain, High Importance",
    "EMI & Installments": "Low Pain, High Importance",
    "Confusing Payment options": "Low Pain, High Importance"
}

In [47]:
# @tool
# def payment_failure(transaction_id: str):
    
#     """Multiply two integers together."""
    

In [48]:
Index = "query-category-new"
pc = pc(api_key=os.environ.get("PINECONE_API_KEY"))

index = pc.Index(Index)

vectorstore = Pinecone(index, embedding=MistralAIEmbeddings())
retriever = vectorstore.as_retriever(k=1)

c:\Users\aichr\.virtualenvs\Amazon-HackOn-vcHtwuFW\lib\site-packages\langchain_mistralai\embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [49]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "payment_query_search",
    "Search for information related to payment queries. For any questions about Payment and payment methods, you must use this tool!",
)

In [50]:
tools = [retriever_tool]

In [51]:
model_with_tools = llm.bind_tools(tools)

In [52]:
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today? If you have any questions about payments or payment methods, I can help you with that.
ToolCalls: []


In [53]:
print(retriever.invoke("Why my payment failure occured ?"))

[Document(page_content='Category: Payment Failure\nQuestion: What are the common reasons for payment failure?\nAnswer:\nCommon reasons for payment failure on Amazon include:\n1. Incorrect card details: Ensure the card number, expiration date, and CVV are entered correctly.\n2. Insufficient funds: Verify that your account has enough funds to complete the transaction.\n3. Expired card: Check if your card has expired and, if so, use a different card.\n4. Bank restrictions: Your bank may decline the transaction for security reasons or due to restrictions on your account. Contact your bank for more information.\n5. Network issues: Temporary network problems can cause payment failures. Wait a few moments and try again.\n6. Payment gateway issues: Sometimes, issues with the payment gateway can result in failures. If this happens, try a different payment method.\nIf none of these apply, please contact Amazon Customer Service for further assistance.', metadata={'category': 'Payment Failure', 'q

In [54]:
response = model_with_tools.invoke([HumanMessage(content="Why my payment failure occured ?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'payment_query_search', 'args': {'query': 'payment failure'}, 'id': 'A4qXWL0Mg'}]


In [55]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [56]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [57]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [58]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!',
 'output': 'Hello! How can I assist you today? If you have any questions about payments or payment methods, I can help you with that using the payment_query_search function.'}

In [ ]:
from langchain import PromptTemplate

# create the prompt
prompt_template: str = """/
You are a customer service agent at Amazon and you have to resolve all payment related issues. You have access to a tool called 'payment_query_search' which you should use 
whenever a user asks a question related to payment errors, methods, or processes. When you receive such a query, call the 'payment_query_search' tool to fetch the information./ 
question: {question}. Do not use technical words, give easy/
to understand responses.
"""

prompt = PromptTemplate.from_template(template=prompt_template)

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    question="Why won't a vehicle start on ignition?")

In [59]:
agent_executor.invoke({"input": "How can I resolve a payment processing error?"})

{'input': 'How can I resolve a payment processing error?',
 'output': 'To help you resolve a payment processing error, I would need more specific information about the error. Please provide me with the error message or code, and any other relevant details. I will then use the payment_query_search function to look up the necessary information to assist you in resolving the error.\n\nPlease be aware that I can only provide information and guidance on resolving payment processing errors. I do not have the ability to directly fix any technical issues or process payments.\n\nHere is an example of how you can provide me with the necessary information:\n\n"I am encountering a payment processing error with the error code \'ABC123\'. The error message states \'There was an issue processing your payment. Please try again later.\' I have tried processing the payment multiple times, but I keep encountering the same error. I am using a Visa credit card and my billing information is correct."\n\nWit